# Shire - List - Accumulations
## Beachten
* Set Columns manually (or you will have 28 columns!)

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import PDF

In [2]:
# Read pdf into DataFrame
columns = [64.64, 204.83, 266.06, 306.34, 369.46, 429.34, 482.47, 545.29, 571.91, 606.53, 622.35, 680.54, 696.86]
df = tabula.read_pdf("report-switzerland-english.pdf", pages='all', lattice=False, columns=columns, guess=False)

## Format Table

In [43]:
df_export = df.copy()

#Fix RnD
name = df_export.columns[1]
total = df_export.columns[13]
df_export.loc[df_export[total].notna() & df_export[name].isna(), name] = 'rnd'
df_export.loc[df_export[name] == 'rnd', df_export.columns[11]] = df_export[df_export.columns[12]]

#Drop Columns
df_export = df_export.drop(df_export.columns[2:6],axis=1)
df_export = df_export.drop(df_export.columns[[0, 8]],axis=1)

#Rename
df_export.columns = fix_columns_accumulations[0: 8]

# Select
amount = df_export[df_export.type.str.contains('Aggregate amount', na=False)]
recipients = df_export[df_export.type.str.contains('Number of Recipients', na=False)]
rnd = df_export[df_export.type.str.contains('rnd', na=False)]

df_export = pd.concat([amount, recipients, rnd])

#Cleanup Numbers
df_export = remove_comma(df_export)
df_export = amounts_to_number(df_export)

#Rename first column
df_export = df_export.reset_index(drop=True)
df_export.iloc[0,0] = "hcp_amount"
df_export.iloc[1,0] = "hco_amount"
df_export.iloc[2,0] = "hcp_count"
df_export.iloc[3,0] = "hco_count"
df_export.iloc[4,0] = "rnd"
df_export.loc[df_export.type == 'rnd', "total"] = df_export['related_expenses']
df_export.loc[df_export.type == 'rnd', "related_expenses"] = np.NaN

#Export
export_acumulations(df_export, 'shire')

Total nicht Summe der Werte
saved


In [ ]:
#write_to_csv(df_export, 'tmp.csv', True)
#write_to_excel(df_export, 'tmp.xlsx')